1. Construyan la base de datos trimestral. Esto es, la observación para un trimestre de una variable mensual debería ser el promedio de los tres meses que conforman el trimestre.

In [1]:
#Importamos los módulos
import numpy as np    # Algebra matricial
from numpy import ndarray, dot
import datetime as dt
import pandas as pd
import random      # Simulacion aleatoria
import matplotlib.pyplot as plt   # gráficos
import os
import sys
from sys import stdout
from scipy.optimize import minimize
import VARstuff      #Funciones BVAR
from math import pi, exp, gamma, sqrt, log, floor    #Funciones matemáticas
from numpy import r_,c_
from numpy.random import normal
random.seed( 0 )     # Fijando la semilla (reproducir resultados)
np.random.seed( 0 )  # Fijando la semilla (reproducir resultados)

In [2]:
from numpy import triu, diag, sqrt, asarray, arange
from numpy import zeros, ones, std, real, log, vstack
from numpy import eye, pi, hstack, concatenate, reshape
from numpy import kron, flipud, isinf, full, nan, dot
from numpy import sum as suma
from numpy.linalg import lstsq, inv, eigvalsh, cholesky
from scipy.special import gammaln
from numpy.random import normal, chisquare, randint

In [3]:
# Importar la data 
data = pd.read_excel('DBmf_sa.xlsx', sheet_name='Final')
data.head()

,Unnamed: 0,ysoc,psoc,rsoc,ti,cc,yp,y,cons,inv,p,r,rer
0,2003-01-01,NaN,442.46892,1.235161,407.313034,NaN,441.90399,432.01587,NaN,NaN,442.04905,3.75,460.250462
1,2003-02-01,NaN,442.85502,1.262143,408.505154,NaN,442.49036,432.55087,NaN,NaN,442.41339,3.80,460.056402
2,2003-03-01,439.984236,443.02107,1.252903,404.758172,-3.571477,440.54751,432.71084,1054.997,912.86808,443.16189,3.82,459.184439
3,2003-04-01,NaN,443.18062,1.258000,402.197974,NaN,446.28762,434.13011,NaN,NaN,443.11822,3.84,459.499765
4,2003-05-01,NaN,443.29447,1.259032,406.623817,NaN,442.88559,434.17208,NaN,NaN,443.19648,3.78,461.494893


In [4]:
#Armando la data trimestral
data['date'] = pd.to_datetime(data['Unnamed: 0'])
data = data.set_index('date') 
df_final = data.resample('Q').mean()

#Resampleando data mensual
data_m = data.resample('M').mean()

df_final.head()

,Unnamed: 0,ysoc,psoc,rsoc,ti,cc,yp,y,cons,inv,p,r,rer
date,,,,,,,,,,,,,
2003-03-31,2003-01-31 00:00:00,439.984236,442.781670,1.250069,406.858787,-3.571477,441.647287,432.425860,1054.9970,912.86808,442.541443,3.790000,459.830435
2003-06-30,2003-05-01 08:00:00,440.745928,443.293357,1.246566,405.092827,-1.021831,444.149057,434.516213,1056.3846,915.11017,443.039737,3.786667,460.931122
2003-09-30,2003-08-01 00:00:00,442.132382,443.853317,1.016774,410.020678,-1.180733,443.289327,435.404730,1057.3694,915.89833,442.825400,3.083333,461.551436
2003-12-31,2003-10-31 16:00:00,443.529388,444.680793,0.996731,415.568779,-0.736083,441.648437,433.851750,1058.3998,912.37232,443.794150,2.606667,463.699389
2004-03-31,2004-01-31 08:00:00,444.459400,445.485807,1.001984,422.829469,-0.427388,445.635123,436.287910,1059.1829,918.40011,445.490713,2.470000,464.065964


In [5]:
#Reordenando la data de tal manera que las primeras variables sean aquellas con la frecuencia mas baja (trimestral).
df_final = df_final.reindex(columns=['ysoc','psoc','rsoc','ti','cc','yp', 'y','cons','inv','p','r','rer'])
#Reordenando data mensual
data_m = data_m.reindex(columns=['ysoc','psoc','rsoc','ti','cc','yp', 'y','cons','inv','p','r','rer'])
df_final.head()

,ysoc,psoc,rsoc,ti,cc,yp,y,cons,inv,p,r,rer
date,,,,,,,,,,,,
2003-03-31,439.984236,442.781670,1.250069,406.858787,-3.571477,441.647287,432.425860,1054.9970,912.86808,442.541443,3.790000,459.830435
2003-06-30,440.745928,443.293357,1.246566,405.092827,-1.021831,444.149057,434.516213,1056.3846,915.11017,443.039737,3.786667,460.931122
2003-09-30,442.132382,443.853317,1.016774,410.020678,-1.180733,443.289327,435.404730,1057.3694,915.89833,442.825400,3.083333,461.551436
2003-12-31,443.529388,444.680793,0.996731,415.568779,-0.736083,441.648437,433.851750,1058.3998,912.37232,443.794150,2.606667,463.699389
2004-03-31,444.459400,445.485807,1.001984,422.829469,-0.427388,445.635123,436.287910,1059.1829,918.40011,445.490713,2.470000,464.065964


2. Modifiquen Hyper ML para incluir un argumento de entrada adicional (llamado ‘flag_sc’), tal que cuando flag_sc = True, se añada el prior de suma de coeficiente (la versión actual solo implementa el prior de Minnesota). Es probable que necesiten añadir más argumentos para que la función logre calcular el prior de suma de coeficientes. Fijen el hiperparámetro de este segundo prior en $\tau=\alpha\lambda$, donde $\lambda$ es el hiperparámetro de ajuste del prior de Minnesota. Entonces $\theta=[\lambda,\alpha]'$.

In [54]:
# def Hyper_ML(theta, X, y, sigmas, deltas, mus, flag_sc = False):
#     T,k  = y.shape
#     n,klagp1 = X.shape
#     lag = (klagp1-1)//k
#     lmbda = theta[0]
    
#     yd, Xd, T_l = VARstuff.get_DumObsLitterman(lmbda, deltas, sigmas, lag)
# #     Text = T + Td #Esta primera parte se tiene que modificar para la tarea 
#     Td_sc=0     # Td_sc es cero debido a que solo calculamos la prior de Litterman
    
#     #Si se cumple flag_sc hallar Litterman y Suma de coeficientes:
#     if flag_sc:
#         alpha=theta[1]
#         tau=alpha*lmbda
#         yd_sc, Xd_sc, Td_sc= VARstuff.get_DumObsSumCoef(tau, mus, sigmas, lag) #Generar las dummys de la prior de Suma de coeficientes
#         yd = np.r_[yd,yd_sc]
#         Xd = np.r_[Xd,Xd_sc]
        
#     T, k = X.shape
#     Td = T_l + Td_sc
#     Text = T + Td  
    
#     constants = -((k*T)/2)*log(pi) + suma( gammaln( (Text-n+1-(1+arange(k)))/2) ) - suma( gammaln( (Td-n+1-(1+arange(k)))/2) )
#     Omegaprior = diag(1/diag(dot(Xd.T,Xd)))
#     Uprior = dot((eye(Td)-dot(Xd,dot(Omegaprior,Xd.T))),yd)
#     Sprior = diag(diag(dot(Uprior.T,Uprior)))
#     Sprior_logdet = suma(log(diag(Sprior)))
#     yext = vstack((yd, y))
#     Xext = vstack((Xd, X))
#     iOmegapost = dot(Xext.T,Xext)
#     Omegapost = inv(iOmegapost)
#     Upost      = dot((eye(Text)-dot(Xext,dot(Omegapost,Xext.T))),yext)
#     Spost      = dot(Upost.T,Upost)
    
#     LiS     = diag(1./sqrt(diag(Sprior)))
#     LOmega  = cholesky(Omegaprior)
#     M       = Spost-Sprior
#     LXL     = dot(LOmega.T,dot(dot(X.T,X),LOmega))
#     LML     = dot(LiS.T,dot(M,LiS))
#     eigLXL  = real(eigvalsh(LXL))
#     eigLML  = real(eigvalsh(LML))
#     LXLp1_detlog = suma(log(1+eigLXL))
#     LMLp1_detlog = suma(log(1+eigLML))
    
#     resto   = -(T/2)*Sprior_logdet - (k/2)*LXLp1_detlog - ((Text-n)/2)*LMLp1_detlog
        
#     logML = constants*0 + resto
    
#     return -logML

In [64]:
def Hyper_ML(theta, X, y, deltas, sigmas, lag, flag_sc, mus):
    T,k  = y.shape
    n,klagp1 = X.shape
    lag = (klagp1-1)//k
    lmbda = theta[0]
    alpha = theta[1]
    tau = lmbda * alpha
    
    if flag_sc:
        yd, Xd, Td = get_DumObsLitterman(lmbda, deltas, sigmas, lag)
        yd1,Xd1,Td1 = get_DumObsSumCoef(tau, sigmas, mus, lag)
        Xd = np.r_[Xd,Xd1]
        yd = np.r_[yd,yd1]
        Td=Td+Td1
        Text = T + Td 
    
    else:
        yd, Xd, Td = get_DumObsLitterman(lmbda, deltas, sigmas, lag)
        Text = T + Td 
    
    constants = -((k*T)/2)*log(pi) + suma( gammaln( (Text-n+1-(1+arange(k)))/2) ) - suma( gammaln( (Td-n+1-(1+arange(k)))/2) )
    Omegaprior = diag(1/diag(dot(Xd.T,Xd)))
    Uprior = dot((eye(Td)-dot(Xd,dot(Omegaprior,Xd.T))),yd)
    Sprior = diag(diag(dot(Uprior.T,Uprior)))
    Sprior_logdet = suma(log(diag(Sprior)))
    yext = vstack((yd, y))
    Xext = vstack((Xd, X))
    iOmegapost = dot(Xext.T,Xext)
    Omegapost = inv(iOmegapost)
    Upost      = dot((eye(Text)-dot(Xext,dot(Omegapost,Xext.T))),yext)
    Spost      = dot(Upost.T,Upost)
    
    LiS     = diag(1./sqrt(diag(Sprior)))
    LOmega  = cholesky(Omegaprior)
    M       = Spost-Sprior
    LXL     = dot(LOmega.T,dot(dot(X.T,X),LOmega))
    LML     = dot(LiS.T,dot(M,LiS))
    eigLXL  = real(eigvalsh(LXL))
    eigLML  = real(eigvalsh(LML))
    LXLp1_detlog = suma(log(1+eigLXL))
    LMLp1_detlog = suma(log(1+eigLML))
    
    resto   = -(T/2)*Sprior_logdet - (k/2)*LXLp1_detlog - ((Text-n)/2)*LMLp1_detlog
        
    logML = constants + resto

    return -logML

**Pregunta 3**

Encuentren los hiperparámetros que maximizan la verosimilitud. En estas regresiones, utilice 5 rezagos (p = 5) dada la frecuencia trimestral.

In [65]:
T = df_final.ysoc.size #número de periodos, comando .size
T

68

In [66]:
H = 4 #se separa la muestra en una para estimar y otra para comparar eliminando 12 observaciones (se proyectará con bvar 12 periodos en adelante)
df_estimate = df_final[0:(T-H)].copy() #notar que el último elemento se traslapa
df_eval = df_final[(T-H-1):T].copy()

In [67]:
df_estimate

,ysoc,psoc,rsoc,ti,cc,yp,y,cons,inv,p,r,rer
date,,,,,,,,,,,,
2003-03-31,439.984236,442.781670,1.250069,406.858787,-3.571477,441.647287,432.425860,1054.9970,912.86808,442.541443,3.790000,459.830435
2003-06-30,440.745928,443.293357,1.246566,405.092827,-1.021831,444.149057,434.516213,1056.3846,915.11017,443.039737,3.786667,460.931122
2003-09-30,442.132382,443.853317,1.016774,410.020678,-1.180733,443.289327,435.404730,1057.3694,915.89833,442.825400,3.083333,461.551436
2003-12-31,443.529388,444.680793,0.996731,415.568779,-0.736083,441.648437,433.851750,1058.3998,912.37232,443.794150,2.606667,463.699389
2004-03-31,444.459400,445.485807,1.001984,422.829469,-0.427388,445.635123,436.287910,1059.1829,918.40011,445.490713,2.470000,464.065964
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31,492.947531,480.295980,1.204161,463.154376,-1.125857,497.369513,508.836507,1133.2353,1029.22780,484.893080,3.370700,458.106989
2018-03-31,493.931306,480.922623,1.446048,462.135858,-2.608607,499.339327,510.443247,1134.2663,1031.45050,485.220233,2.971633,460.946729
2018-06-30,494.695985,481.381587,1.737333,458.559709,-1.420272,503.431087,512.123073,1135.6936,1030.87030,485.497043,2.764833,460.006888


In [68]:
df_eval

,ysoc,psoc,rsoc,ti,cc,yp,y,cons,inv,p,r,rer
date,,,,,,,,,,,,
2018-12-31,496.004207,482.453143,2.219867,453.129609,-0.438509,502.555080,513.094130,1137.1125,1033.6577,486.938717,2.758767,457.913783
2019-03-31,496.860391,482.976810,2.401720,453.335762,-3.158419,498.219253,513.089207,1137.8190,1033.4926,487.324557,2.752533,458.019001
2019-06-30,497.626638,483.663010,2.397434,455.084251,-1.510254,498.963870,513.382100,1138.4729,1036.5675,488.002723,2.765033,456.812725
2019-09-30,498.125880,484.296063,2.190570,456.423200,-1.661783,501.499457,514.499260,1139.3814,1036.1511,488.279390,2.606900,456.434243
2019-12-31,498.466838,485.083963,1.644659,454.946193,-0.053579,503.146797,514.802380,1140.0602,1032.6102,488.803713,2.351000,457.144118


Preparando datos para procedimientos bayesianos

In [69]:
lag = 5
k = df_final.columns.size
data2est = df_estimate.values #valores de la muestra de estimación
y,X,k,Tf = VARstuff.get_yXform(data2est,lag) #obtienes las matrices necesarias para estimar MCO dada la data y el número de rezagos que deseas
# k variables, tf periodos
y2e = df_eval.values #muestra a usar para comparar con el pronóstico .values para que lo saque como array

In [70]:
X.shape

(59, 61)

In [71]:
sigmas = VARstuff.get_sigmas(data2est,lag)
deltas = np.array([1, 1, 0, 1, 0, 1, 1 ,1 ,1 ,1 ,0 ,1])
mus = np.mean(data2est,0)
MinPos = 10**-6
bnds = ((MinPos,0.3-MinPos),(MinPos,None))

In [72]:
from scipy.optimize import minimize
theta0 = np.array([0, 0])
# res = minimize(Hyper_ML, theta0, args = (X, y, sigmas, deltas, mus, True), method='SLSQP', options={'disp': True}, bounds=bnds)

TypeError: Hyper_ML() missing 1 required positional argument: 'mus'

In [63]:
res.x

array([1.e-06, 1.e-06])

In [43]:
Hyper_ML(theta0, X, y, sigmas, deltas, mus, flag_sc = True)

C:\Users\cursobcrp\Desktop\BCRP 2024\3. Inferencia Bayesiana - Ledesma\Tareas\TAREA 2\VARstuff.py:364: RuntimeWarning: divide by zero encountered in divide
  yd = diag(sd) / lmbda
C:\Users\cursobcrp\Desktop\BCRP 2024\3. Inferencia Bayesiana - Ledesma\Tareas\TAREA 2\VARstuff.py:364: RuntimeWarning: invalid value encountered in divide
  yd = diag(sd) / lmbda
C:\Users\cursobcrp\Desktop\BCRP 2024\3. Inferencia Bayesiana - Ledesma\Tareas\TAREA 2\VARstuff.py:368: RuntimeWarning: divide by zero encountered in divide
  Xd = vstack([kron(Jp, diag(sigmas)/lmbda), zeros((k+1, lag*k))])
C:\Users\cursobcrp\Desktop\BCRP 2024\3. Inferencia Bayesiana - Ledesma\Tareas\TAREA 2\VARstuff.py:368: RuntimeWarning: invalid value encountered in divide
  Xd = vstack([kron(Jp, diag(sigmas)/lmbda), zeros((k+1, lag*k))])
C:\Users\cursobcrp\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
C:\

LinAlgError: Matrix is not positive definite